In [1]:
from Func import Dissertation_Func_1D

In [2]:
# ==== Imports ====
import os, json
from pathlib import Path
import numpy as np
from joblib import Parallel, delayed

# (Assumes Dissertation_Func_1D class is already defined in scope, exactly as you posted.)

# ==== Helpers for saving ====
def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def _atomic_write_json(path: Path, obj):
    tmp = path.with_suffix(".tmp")
    with open(tmp, "w") as f:
        json.dump(obj, f, indent=2)
    os.replace(tmp, path)

def _save_summary(run_dir: Path, meta: dict):
    with open(run_dir / "summary.json", "w") as f:
        json.dump(meta, f, indent=2)

def _save_fronts(run_dir: Path, t_fronts, x_fronts):
    np.savez_compressed(run_dir / "fronts.npz",
                        t_fronts=np.asarray(t_fronts),
                        x_fronts=np.asarray(x_fronts))

def _save_snapshots_every_150(run_dir: Path, model):
    # every 150th step, plus the final step
    idx = np.unique(np.concatenate([
        np.arange(0, model.Nt, 150),
        np.array([model.Nt - 1])
    ]))
    np.savez_compressed(
        run_dir / "snapshots.npz",
        x=model.x,
        times=model.times[idx],
        N_arr=model.N_arr[idx, :],
        M_arr=model.M_arr[idx, :]
    )

def _fmt_val(v):
    # Compact but stable folder names for floats (keeps integers clean)
    if isinstance(v, (int, np.integer)) or (isinstance(v, float) and v.is_integer()):
        return f"{int(v)}"
    return f"{v}".rstrip('0').rstrip('.') if '.' in f"{v}" else f"{v}"

# ==== Worker with failure logging (returns c and r2) ====
def run_one(lam, alpha, m0,
            base_dir="speeds_func",
            model_kwargs=None,
            overwrite=False):
    if model_kwargs is None:
        model_kwargs = {}

    try:
        base = Path(base_dir)
        lam_dir = base / f"lambda_{_fmt_val(lam)}"
        alpha_dir = lam_dir / f"alpha_{_fmt_val(alpha)}"
        run_dir = alpha_dir / f"m0_{_fmt_val(m0)}"
        _ensure_dir(run_dir)

        # Skip if exists (unless overwrite)
        if not overwrite and (run_dir / "summary.json").exists():
            return ("skipped", lam, alpha, m0)

        # Avoid thread over-subscription
        os.environ.setdefault("OMP_NUM_THREADS", "1")
        os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
        os.environ.setdefault("MKL_NUM_THREADS", "1")
        os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

        # Build and run model (key change: pass alpha)
        model = Dissertation_Func_1D(k=lam, alpha=alpha, m0=m0, **model_kwargs)
        model.solve()

        # Estimate wave speed on u (N)
        c, b, r2 = model.estimate_wave_speed(
            threshold=0.5, band=(0.1, 0.9), spline_type='cubic',
            plot=False, target='N'
        )

        # Treat missing/NaN speed as a failure
        if (c is None) or (isinstance(c, float) and np.isnan(c)):
            raise ValueError("Wave speed could not be calculated")

        model.wave_speed = c

        # Front tracking (save raw points)
        t_fronts, x_fronts = model.track_wavefront_local_interpolation(
            threshold=0.5, band=(0.1, 0.9),
            spline_type='cubic', target='N'
        )

        # Save artifacts
        _save_summary(run_dir, dict(
            lambda_val=float(lam),
            alpha=float(alpha),
            m0=float(m0),
            wave_speed=float(c),
            r2=(float(r2) if r2 is not None else None),
            dt=model.dt, T=model.T, L=model.L, N=model.N,
            init_type=model.init_type,
            t_start=model.t_start, t_end=model.t_end,
            saved_stride=150
        ))
        _save_fronts(run_dir, t_fronts, x_fronts)
        _save_snapshots_every_150(run_dir, model)

        return ("done", lam, alpha, m0, float(c), (float(r2) if r2 is not None else None))

    except Exception as e:
        return ("failed", lam, alpha, m0, str(e))

# ==== Parallel grid runner with failure list AND low-R² list ====
def run_grid(lambda_vals, alpha_vals, m0_vals,
             base_dir="speeds_func",
             model_kwargs=None,
             overwrite=False,
             n_jobs=-1, verbose=10):

    if model_kwargs is None:
        model_kwargs = {}

    tasks = [(lam, alpha, m0) for lam in lambda_vals for alpha in alpha_vals for m0 in m0_vals]
    results = Parallel(n_jobs=n_jobs, verbose=verbose, backend="loky")(
        delayed(run_one)(
            lam, alpha, m0,
            base_dir=base_dir,
            model_kwargs=model_kwargs,
            overwrite=overwrite
        ) for lam, alpha, m0 in tasks
    )

    done, skipped, failed, low_r2 = [], [], [], []
    for r in results:
        tag = r[0]
        if tag == "done":
            # r = ("done", lam, alpha, m0, c, r2)
            _, lam, alpha, m0, c, r2 = r
            done.append({"lambda": lam, "alpha": alpha, "m0": m0, "c": c, "r2": r2})
            if (r2 is None) or (isinstance(r2, float) and (np.isnan(r2) or r2 < .999)):
                low_r2.append({"lambda": lam, "alpha": alpha, "m0": m0, "c": c, "r2": r2})
        elif tag == "skipped":
            _, lam, alpha, m0 = r
            skipped.append({"lambda": lam, "alpha": alpha, "m0": m0})
        elif tag == "failed":
            _, lam, alpha, m0, msg = r
            failed.append({"lambda": lam, "alpha": alpha, "m0": m0, "error": msg})

    # Save failure and low-R² lists for later
    base = Path(base_dir)
    _ensure_dir(base)
    _atomic_write_json(base / "failed_runs.json", failed)
    _atomic_write_json(base / "low_r2_runs.json", low_r2)

    # Console summary
    print(f"✅ Done: {len(done)}, Skipped: {len(skipped)}, Failed: {len(failed)}, Low-R²: {len(low_r2)}")
    if failed:
        print("❌ Failed runs:")
        for item in failed[:20]:  # limit console spam
            print(f"  lambda={item['lambda']}, alpha={item['alpha']}, m0={item['m0']} | error: {item['error']}")
    if low_r2:
        print("⚠️  Low-R² runs (R² < 1):")
        for item in low_r2[:20]:
            print(f"  λ={item['lambda']}, α={item['alpha']}, m0={item['m0']} | c={item['c']} | r2={item['r2']}")

    return {"done": done, "skipped": skipped, "failed": failed, "low_r2": low_r2}

In [3]:
# ==== Your parameter lists ====
lambda_vals = [0.001, 0.01, 0.1, 0.5, 1, 5, 10]
alpha_vals  = [0.001, 0.01, 0.1, 0.5, 1, 5, 10]

m0_vals = [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5,
           0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]

shared_kwargs = dict(
    L=200, N=5001, T=200, dt=0.1,
    init_type="tanh", steepness=0.85, perc=0.4,
    t_start=60, t_end=180, num_points=200,
    n0=1.0, K=1.0, rho=1.0, D=1.0, Mmax=1.0
    # alpha is passed per-task, not here
)

# ==== Run and capture results ====
results = run_grid(lambda_vals, alpha_vals, m0_vals,
                   base_dir="speeds_func",
                   model_kwargs=shared_kwargs,
                   overwrite=False,
                   n_jobs=-1, verbose=10)

# ==== Print flagged results ====
print("\n=== Summary of Problematic Runs ===")
flagged = []

# Failed runs
for f in results["failed"]:
    flagged.append({"lambda": f["lambda"], "alpha": f["alpha"], "m0": f["m0"], "reason": f"FAIL: {f['error']}"})

# Load summaries for "done" runs and check speed/R²
for d in results["done"]:
    summary_path = Path("speeds_func") / f"lambda_{_fmt_val(d['lambda'])}" / f"alpha_{_fmt_val(d['alpha'])}" / f"m0_{_fmt_val(d['m0'])}" / "summary.json"
    if summary_path.exists():
        with open(summary_path, "r") as f:
            meta = json.load(f)
        c = meta.get("wave_speed", None)
        r2 = meta.get("r2", None)

        if c is None:
            flagged.append({"lambda": d["lambda"], "alpha": d["alpha"], "m0": d["m0"], "reason": "NO SPEED"})
        elif (r2 is None) or (isinstance(r2, float) and (np.isnan(r2) or r2 < 0.999)):
            flagged.append({"lambda": d["lambda"], "alpha": d["alpha"], "m0": d["m0"], "reason": f"LOW R² ({r2})"})

# Print list
if flagged:
    for item in flagged:
        print(f"λ={item['lambda']}, α={item['alpha']}, m0={item['m0']} -> {item['reason']}")
else:
    print("✅ No failures, no low R², and all speeds calculated.")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Estimated speed = 0.8895, R² = 1.0000
Estimated speed = 0.7013, R² = 0.9990
Estimated speed = 0.7751, R² = 0.9997
Estimated speed = 0.6026, R² = 0.9966
Estimated speed = 0.9977, R² = 1.0000
Estimated speed = 0.6258, R² = 0.9974
Estimated speed = 0.9604, R² = 1.0000
Estimated speed = 0.9951, R² = 1.0000


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   38.1s


Estimated speed = 0.9676, R² = 1.0000


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   59.7s


...

[Parallel(n_jobs=-1)]: Done 669 tasks      | elapsed: 244.7min


Estimated speed = 0.2383, R² = 0.9984
Estimated speed = 0.2343, R² = 0.9994
Estimated speed = 0.2318, R² = 0.9997
Estimated speed = 0.2301, R² = 0.9998
Estimated speed = 0.2277, R² = 1.0000
Estimated speed = 0.2261, R² = 1.0000
Estimated speed = 0.2327, R² = 1.0000
Estimated speed = 0.2327, R² = 1.0000
Estimated speed = 0.2327, R² = 1.0000
Estimated speed = 0.2327, R² = 1.0000
Estimated speed = 0.2327, R² = 1.0000
Estimated speed = 0.2326, R² = 1.0000
Estimated speed = 0.2326, R² = 1.0000
Estimated speed = 0.2325, R² = 1.0000
Estimated speed = 0.2325, R² = 1.0000
Estimated speed = 0.2324, R² = 1.0000
Estimated speed = 0.2324, R² = 1.0000
Estimated speed = 0.2325, R² = 1.0000
Estimated speed = 0.2325, R² = 1.0000
Estimated speed = 0.2325, R² = 1.0000
Estimated speed = 0.2325, R² = 1.0000
Estimated speed = 0.2352, R² = 1.0000
Estimated speed = 0.2352, R² = 1.0000
Estimated speed = 0.2352, R² = 1.0000
Estimated speed = 0.2352, R² = 1.0000
Estimated speed = 0.2352, R² = 1.0000
Estimated sp

[Parallel(n_jobs=-1)]: Done 706 tasks      | elapsed: 246.3min


Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2392, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2391, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated speed = 0.2390, R² = 1.0000
Estimated sp

[Parallel(n_jobs=-1)]: Done 735 out of 735 | elapsed: 247.5min finished


In [5]:
100/ 10000

0.01